In [ ]:
#----------------------------------------------------
# Code to select 3.3-micron PAH emitters
#
#              Jimena Rodriguez
#                    2024
#---------------------------------------------------
# Import Functions
#---------------------------------------------------
from astropy.io import ascii
import matplotlib.pyplot as plt
import numpy as np

#----------------------------------------------------
#List of PHANGS-JWST cycle 1 galaxies
#----------------------------------------------------

galaxy_names=['ngc5068','ic5332','ngc628','ngc3351','ngc3627','ngc2835','ngc4254','ngc4321','ngc4535','ngc1087','ngc4303',
          'ngc1385','ngc1566','ngc1433','ngc7496','ngc1512','ngc1300','ngc1672','ngc1365']

#----------------------------------------------------
# Read the files with the 3 sigma and 5 sigma color dispersion curve parameters
#----------------------------------------------------
fit_table3=ascii.read('..../ComplementaryData/Coef_dispersion_color_curve0_3sigma.csv') # your path to this files
fit_table5=ascii.read('.../ComplementaryData/Coef_dispersion_color_curve0_5sigma.csv')  # your path to this files

#----------------------------------------------------
# Read file with the F335M 5 sigma detection limit 
# These limits were computed using randomly positioned apertures, as explained in Rodriguez+25 
#----------------------------------------------------
mag_lim=ascii.read('..../ComplementaryData/Table_335_300_200_five_sigma.csv')  # your path to this files

#Apertures Correction in F335M and F300M
# used in Rodriguez+25
#----------------------------------------------------
ac_F335M=-0.66
ac_F300M=-0.68


for galaxy in galaxy_names:
    

    #----------------------------------------------------    
    # Define parameters for selecting 3.3µm PAH emitters  
    #----------------------------------------------------
    a3=fit_table3['coef_a'][fit_table3['galaxy']==galaxy]
    b3=fit_table3['coef_b'][fit_table3['galaxy']==galaxy]
    c3=fit_table3['coef_c'][fit_table3['galaxy']==galaxy]
    a5=fit_table5['coef_a'][fit_table5['galaxy']==galaxy]
    b5=fit_table5['coef_b'][fit_table5['galaxy']==galaxy]
    c5=fit_table5['coef_c'][fit_table5['galaxy']==galaxy]
    F335_lim=mag_lim['F335M_ab_4p'][mag_lim['galaxy']==galaxy]-ac_F335M 
    F300_lim=mag_lim['F300M_ab_4p'][mag_lim['galaxy']==galaxy]-ac_F300M 
    
    
    if galaxy=='ngc5068':
        magnitudes=np.linspace(20,30,20)
    else:
        magnitudes=np.linspace(20,28,20)
        
    if galaxy=='ngc628':
        galaxy_p='ngc0628'
    else:
        galaxy_p=galaxy
    
    
    #----------------------------------------------------
    # 'table' file: Contains the photometry of all sources detected in F335M.
    #----------------------------------------------------
    table=ascii.read('..../Phot_v5p4_src_335_'+galaxy+'.csv') # your path to the photometry table 

    #----------------------------------------------------
    # pop2: Selection of objects with color dispersion between 3σ and 5σ  
    # pop3: Selection of objects with color dispersion greater than 5σ  
    #----------------------------------------------------
    pop2=table[((table['F300M_ab']-table['F335M_ab'])> (a3 * np.exp(b3 * (table['F335M_ab'])) + c3)) & ((table['F300M_ab']-table['F335M_ab'])<=(a5 * np.exp(b5 *(table['F335M_ab'])) + c5))]
    pop3=table[(table['F300M_ab']-table['F335M_ab'])> (a5 * np.exp(b5 * table['F335M_ab']) + c5)]
    
    #----------------------------------------------------
    # Apply F300M and F335M magnitude cuts 
    #----------------------------------------------------
    pop2_lim=pop2[(pop2['F335M_ab']<F335_lim) & (pop2['F300M_ab']<F300_lim)]
    pop3_lim=pop3[(pop3['F335M_ab']<F335_lim) & (pop3['F300M_ab']<F300_lim)]
    
    #----------------------------------------------------
    # Save files
    #----------------------------------------------------
    pop2_lim.write('.../population2_'+galaxy+'.csv', overwrite=True, format='ascii.csv') # path to your output folder
    pop3_lim.write('.../population3_'+galaxy+'.csv', overwrite=True, format='ascii.csv') # path to your output folder